# Transfering Data to Snowflake

In [20]:
import os
import json
from pathlib import Path
import geopandas as gpd
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import StructType, StructField, IntegerType, StringType, FloatType, BooleanType
import pandas as pd

params = json.loads(os.environ.get("SNOWFLAKE_CONNECTION"))
session = Session.builder.configs(params).create()

In [29]:
GEOJSON_PATH = "ODOT_Data/dim_signal_XD.geojson"
TABLE = "XD_GEOM"

gdf = gpd.read_file(GEOJSON_PATH)
# ensure lon/lat WGS84
gdf = gdf.to_crs(epsg=4326)
# convert geometry to WKT for sql query
gdf['WKT'] = gdf['geometry'].apply(lambda g: g.wkt)
gdf = gdf[['XD', 'WKT']]
gdf

,XD,WKT
0,1236851456,POLYGON ((-123.84950293892057 46.1909069870806...
1,1236985053,POLYGON ((-123.17054104678215 44.1032762677248...
2,448854484,POLYGON ((-124.05812470345623 44.6346007743394...
3,484487649,POLYGON ((-123.19796626496156 45.2122194198096...
4,385980703,POLYGON ((-123.34827999811587 42.4224262637362...
...,...,...
16063,448855840,POLYGON ((-123.04214614172004 44.0768505441109...
16064,1236852335,POLYGON ((-122.83289018085857 45.4476262638553...
16065,448835420,POLYGON ((-122.82601626463054 45.4608324528451...
16066,385766005,POLYGON ((-122.56006557258574 45.4247273686460...


In [32]:
# prepare data
data = [(int(row['XD']), row['WKT']) for _, row in gdf.iterrows()]
schema = StructType([
    StructField("XD", IntegerType()),
    StructField("WKT", StringType())
])

# create Snowpark DataFrame
df = session.create_dataframe(data, schema)

# insert with TO_GEOGRAPHY
from snowflake.snowpark import functions as F
df = df.with_column("GEOM", F.sql_expr("TO_GEOGRAPHY(WKT)")).drop("WKT")
df.write.save_as_table("XD_GEOM", mode="truncate") # overwrite existing table

# Transfer dim_signals data

In [21]:
PATH = "ODOT_Data/dim_signals.parquet"
TABLE = "DIM_SIGNALS"

data = pd.read_parquet(PATH)[['ID', 'District', 'Latitude', 'Longitude', 'ODOT_Maintained', 'Intersection']]

# Add ODOT prefix to District values 1-13 (they're stored as strings)
#data['District'] = data['District'].apply(lambda x: f"_D{x}" if x.isdigit() and 1 <= int(x) <= 13 else x)

# Add Name column (ID - Intersection, but only add Intersection if it is not null)
data['Name'] = data.apply(lambda row: f"{row['ID']} - {row['Intersection']}" if pd.notnull(row['Intersection']) else row['ID'], axis=1)

# Drop Intersection column
data = data.drop(columns=['Intersection'])

# Make all column names uppercase to match Snowflake
data.columns = [col.upper() for col in data.columns]

data

,ID,DISTRICT,LATITUDE,LONGITUDE,ODOT_MAINTAINED,NAME
0,01001,1,46.023964,-123.911674,True,01001 - US101_PACIFIC
1,01005,1,45.979191,-123.926712,True,"01005 - US101_""U""AVE"
2,01006,1,45.993122,-123.920639,True,01006 - US101_BROADWAY
3,01007,1,46.001537,-123.917435,True,01007 - US101_12TH
4,01009,1,46.145222,-123.914268,True,01009 - ENSIGN LANE_WALMART/COSTCO ACCESS
...,...,...,...,...,...,...
3631,Beaverton_99,Beaverton,45.477500,-122.763576,False,Beaverton_99 - Scholls Ferry and Portland Golf...
3632,Beaverton_100,Beaverton,45.471954,-122.773923,False,Beaverton_100 - Scholls Ferry and Allen
3633,Beaverton_104,Beaverton,45.462558,-122.779419,False,Beaverton_104 - Scholls Ferry and Whitford Mid...
3634,Beaverton_105,Beaverton,45.458740,-122.781420,False,Beaverton_105 - Scholls Ferry and TVFR Station 53


In [ ]:
# create Snowpark DataFrame
df = session.create_dataframe(data)

# insert
df.write.save_as_table("DIM_SIGNALS", mode="truncate") # overwrite existing table